In [ ]:
import re
import math

from datetime import datetime, timedelta
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.agents import Tool, create_tool_calling_agent, AgentExecutor
from typing import List, Optional
from pydantic import BaseModel

load_dotenv()

In [ ]:
# --- Tools ---

# --- 1. Mathematical expressions ---
def math_expression_calculator(input: str) -> str:
    if re.search(r"x(\^2|²)?", input, re.IGNORECASE):
        return "Error: Detected variable 'x' or quadratic term. Use the QuadraticEquationSolver for equations with x."
    # Extract everything after a colon, or fallback to whole input
    expr = input.split(":", 1)[-1].strip() if ":" in input else input.strip()
    # Remove all characters except numbers, operators, decimals, and parentheses
    expr = re.sub(r"[^0-9\.\+\-\*\/\(\)]", "", expr)
    if not expr or not re.search(r"\d", expr):
        return "Error: No valid math expression found."
    try:
        result = eval(expr, {"__builtins__": {}})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"
    
# --- 2. Date difference calculator ---
def days_passed_calculation(input: str) -> str:
    match = re.search(r"\d{4}-\d{2}-\d{2}", input)
    if not match:
        return "Error: No date in 'YYYY-MM-DD' format found in input."
    date_str = match.group()
    try:
        user_date = datetime.strptime(date_str, "%Y-%m-%d")
    except ValueError:
        return f"Error: Could not parse '{date_str}' with format '%Y-%m-%d'."
    current_date = datetime.today()
    time_difference: timedelta = current_date - user_date
    days_passed = time_difference.days
    return f"{days_passed} days have passed since {user_date.date()}." # str(days_passed)

# --- 3. Fahrenheit to Celsius conversion tool ---
def fahrenheit_to_celsius(input: str) -> str:
    try:
        match = re.search(r"[-+]?\d*\.?\d+", input)
        if not match:
            return "Error: No valid number found."
        fahrenheit_str = match.group()
        temp_in_fahrenheit = float(fahrenheit_str)
        temp_in_celsius = (temp_in_fahrenheit - 32) * 5.0 / 9.0
        return f"{temp_in_fahrenheit}°F is {temp_in_celsius:.2f}°C."
    except Exception as e:
        return f"Error: {str(e)}"
    
# --- 4. Quadratic equation solver ---
def parse_coefficients(value: str) -> float:
    if value in ["", "+"]:
        return 1.0
    elif value == "-":
        return -1.0
    return float(value)

def quadratic_equation_solver(input: str) -> str:
    if not re.search(r"x(\^2|²)?", input, re.IGNORECASE):
        return "Error: No variable 'x' or quadratic term detected. Use MathExpressionCalculator for pure math expressions."
    # Normalize input
    input = input.lower()
    input = input.replace("=0", "")
    input = re.sub(r"[^\dx\^+\-\.=²]", "", input)
    
    match = re.search(r"([-+]?\d*\.?\d*)x(?:\^2|²)?\s*([-+]\d*\.?\d*)x\s*([-+]\d*\.?\d*)", input.replace(" ", ""))
    if not match:
        return "Error: No valid quadratic equation found. Please provide input like '2x^2 + 3x + 1 = 0'."
    a, b, c = map(parse_coefficients, match.groups())
    if a == 0:
        return "Error: 'a' cannot be zero in a quadratic equation."
    discriminant = b ** 2 - 4 * a * c
    if discriminant > 0:
        root1 = (-b + math.sqrt(discriminant)) / (2*a)
        root2 = (-b - math.sqrt(discriminant)) / (2*a)
        return f"Two real roots exist: {root1:.2f} and {root2:.2f}"
    elif discriminant == 0:
        root = -b / (2*a)
        return f"One real root exists: {root:.2f}"
    else:
        return "Error: No real roots exist for the given quadratic equation."
    
tools = [
    Tool(
        name="Use_this_only_for_math_expressions",
        func=math_expression_calculator,
        description=(
            "Safely evaluate math expressions with only numbers and operators, like '(8 + 2) * (5 - 3) / 2'. "
            "DO NOT use if the input contains any 'x', 'x^2', 'x²', or is an equation to solve. "
            "DO NOT use for dates or temperature conversions."
        ),
        return_direct=True,
    ),
    Tool(
        name="Use_this_to_calculate_days_passed_from_a_date",
        func=days_passed_calculation,
        description=(
            "Use ONLY for questions like 'how many days have passed since 1995-01-01', or date difference questions. "
            "Input must contain a date in 'YYYY-MM-DD' format. "
            "DO NOT use for math expressions, temperature conversions, or equations with x."
        ),
        return_direct=True,
    ),
    Tool(
        name="Use_this_for_fahrenheit_to_celsius_conversion",
        func=fahrenheit_to_celsius,
        description=(
            "Convert Fahrenheit to Celsius. Input must be a valid number representing temperature in Fahrenheit. "
            "DO NOT use for math expressions, dates, or equations with x."
        ),
        return_direct=True,
    ),
    Tool(
        name="Use_this_to_solve_quadratic_equations_with_x_squared",
        func=quadratic_equation_solver,
        description=(
            "ONLY use if the user asks to solve for x or mentions a quadratic equation (with 'x', 'x^2', or 'x²'). "
            "DO NOT use for general math expressions, dates, or temperature conversions."
        ),
        return_direct=True,
    )
]

# --- Prompt Template ---
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful and precise AI assistant. Use the correct tool based on user input.\n\n"
     "Examples:\n"
     "- Input like 'Compute the result of: (10 / 2) + 4 * (1 + 1)' → Use_this_only_for_math_expressions\n"
     "- 'Calculate (8 + 2) * (5 - 3) / 2' → Use_this_only_for_math_expressions\n"
     "- 'What is (9+1)/2?' → Use_this_only_for_math_expressions\n\n"
     "- 'How many days since 1995-01-01?' → Use_this_to_calculate_days_passed_from_a_date\n"
     "- 'Days passed since 2000-01-01?' → Use_this_to_calculate_days_passed_from_a_date\n\n"
     "- 'Convert 98.6 to Celsius' → Use_this_for_fahrenheit_to_celsius_conversion\n"
     "- 'What is 100°F in Celsius?' → Use_this_for_fahrenheit_to_celsius_conversion\n\n"
     "- 'Solve 2x^2 + 3x + 1 = 0' → Use_this_to_solve_quadratic_equations_with_x_squared\n"
     "- 'Find roots of x² - 4x + 4 = 0' → Use_this_to_solve_quadratic_equations_with_x_squared\n\n"
     "Use only one tool per task. NEVER guess or mix tools."),
    ("human", "{input}"),
    ("ai", "{agent_scratchpad}")
])

# --- LLM and Agent Setup ---
def router(user_input):
    if re.search(r"\bx(?:\^2|²)?\b", user_input, re.IGNORECASE) or "solve" in user_input.lower():
        return "quadratic"
    elif re.search(r"\b\d{4}-\d{2}-\d{2}\b", user_input):
        return "days"
    elif re.search(r"(fahrenheit|°f|\bto celsius\b)", user_input.lower()):
        return "temp"
    elif re.search(r"[\d\(\)\+\-\*/]", user_input):
        return "math"
    else:
        return "agent"


llm = ChatOllama(model="llama3.2:1b", temperature=0.0)  # Using Ollama's Llama 3.2 model
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=1,
)

# --- Running the Agent ---
inputs = [
    "Compute the result of: (10 / 2) + 4 * (1 + 1)",
    "Calculate the result of: (8 + 2) * (5 - 3) / 2",
    "How many days have passed since 1995-01-01?",
    "Convert 98.6 to celsius.",
    "Solve the quadratic equation 2x^2 + 3x + 1 = 0"
]

for input_text in inputs:
    route = router(input_text)
    
    if route == "math":
        output = math_expression_calculator(input_text)
    elif route == "days":
        output = days_passed_calculation(input_text)
    elif route == "temp":
        output = fahrenheit_to_celsius(input_text)
    elif route == "quadratic":
        output = quadratic_equation_solver(input_text)
    else:
        output = executor.invoke({"input": input_text})["output"]

    print(f"\n👤 Input: {input_text}\n🤖 Result: {output}")

